In [1]:
####################################
#ENVIRONMENT SETUP

In [2]:
#Importing Libraries
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.ticker as ticker
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import ScalarFormatter
import matplotlib.gridspec as gridspec
import xarray as xr

import sys; import os; import time; from datetime import timedelta
import pickle
import h5py

In [3]:
#MAIN DIRECTORIES
def GetDirectories():
    mainDirectory='/mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/'
    mainCodeDirectory=os.path.join(mainDirectory,"Code/CodeFiles/")
    scratchDirectory='/mnt/lustre/koa/scratch/air673/'
    codeDirectory=os.getcwd()
    return mainDirectory,mainCodeDirectory,scratchDirectory,codeDirectory

[mainDirectory,mainCodeDirectory,scratchDirectory,codeDirectory] = GetDirectories()

In [4]:
#IMPORT CLASSES (from current directory)
sys.path.append(os.path.join(mainCodeDirectory,"Variable_Calculation"))
from CLASSES_Variable_Calculation import ModelData_Class, SlurmJobArray_Class, DataManager_Class

In [5]:
####################################
#LOADING CLASSES

In [ ]:
#data loading class
ModelData = ModelData_Class(mainDirectory, scratchDirectory, simulationNumber=1)
#data manager class
DataManager = DataManager_Class(mainDirectory, scratchDirectory, ModelData.res, ModelData.t_res, ModelData.Nz_str,
                                ModelData.Np_str, dataType="CalculateMoreVariables", dataName="Equivalent_Potential_Temperature",
                                dtype='float32')

In [7]:
#JOB ARRAY SETUP
UsingJobArray=True

def GetNumJobs(res):
    if res=='1km':
        num_jobs=20
    elif res=='250m': 
        num_jobs=100
    return num_jobs
num_jobs = GetNumJobs(ModelData.res)
SlurmJobArray = SlurmJobArray_Class(total_elements=ModelData.Ntime, num_jobs=num_jobs, UsingJobArray=UsingJobArray)
start_job = SlurmJobArray.start_job; end_job = SlurmJobArray.end_job

def GetNumElements():
    num_elements = np.arange(ModelData.Ntime)[start_job:end_job]
    return num_elements
num_elements = GetNumElements()


Running timesteps from 0:7 



In [8]:
####################################
#FUNCTIONS

In [9]:
#Calculation Functions
def GetInputVariables(inputDataDirectory, timeString):    
    
    ################################# PRESSURE VARIABLES
    p0=1e5
    P = DataManager.GetTimestepData(inputDataDirectory, timeString, variableName="prs")
    
    ################################# MIXING RATIO
    rv = DataManager.GetTimestepData(inputDataDirectory, timeString, variableName="qv")
    qc_data = DataManager.GetTimestepData(inputDataDirectory, timeString, variableName="qc")
    qr_data = DataManager.GetTimestepData(inputDataDirectory, timeString, variableName="qr")
    rt=(rv+qc_data+qr_data) #total mixing ratio
    
    ################################# THERMODYNAMICS
    #CONSTANTS (either from CM1 model constants or Emanuel 1994)
    Rd=287.04; Rv=461.5
    Cpd=1005.7 #+-2.5
    Cpv=1870 #+-25
    Cpl=4190 #+-30 #Cl=Cpl=Cvl (same as constant pressure or volume)
    Cpi=2106.0 #+-6 #Ci=Cpi=Cvi (same as constant pressure or volume)

    #LATENT HEATS (From Emanuel 1994)
    Lv0=2.501e6
    Lf0 = 0.3337e6  # latent heat of fusion at 0°C
    Ls0 = 2.834e6 #+-0.01 (inside coefficient) (between -100 and 0 deg C)
    def Lv(T): #Kirchoff's formula L_i,ii= L_i,ii0+(Cpii-Cpi)*(T-273.15)
        Llv=Lv0+(Cpv-Cpl)*(T-273.15) #should it be Cpl. is Cl the same?***
        return Llv
    def Lf(T): #Kirchoff's formula L_i,ii= L_i,ii0+(Cpii-Cpi)*(T-273.15)
        Lil = Lf0 + (Cpl - Cpi) * (T - 273.15) 
        return Lil
    # def Ls(T):
    #     Ls = Lf(T) + Lv(T) #ice to liquid ==> liquid to vapor
    #     return Ls
    def Ls(T): #Kirchoff's formula L_i,ii= L_i,ii0+(Cpii-Cpi)*(T-273.15)
        Liv = Ls0 + (Cpv - Cpi) * (T - 273.15) 
        return Liv
    
    ################################# TEMPERATURE
    theta=DataManager.GetTimestepData(inputDataDirectory, timeString, variableName="th")
    T=theta*(P/p0)**(Rd/Cpd)
    
    ################################# RELATIVE HUMIDITY
    eps=0.622
    #rv=eps*(e/(P-e)) ==> e = rv*P/(eps+rv)
    e=rv*P/(rv+eps)
    Pd=P-e #P=Pd+e ==> Pd=P-e

    #CALCULATING SATURATION VAPOR PRESSURE and RH OVER LIQUID
    def SaturationVaporPressure(T):
        e_s0=611
        T0=273.15
        inner=(Lv(T)/Rv)*((1/T0)-(1/T))
        e_s=e_s0*np.exp(inner)
        return e_s
    e_s = SaturationVaporPressure(T)
    H_vapor = e/e_s 

    #CALCULATING SATURATION VAPOR PRESSURE and RH OVER ICE
    def IceSaturationVaporPressure(T):
        e_si0=611
        T0=273.15
        inner=(Ls(T)/Rv)*((1/T0)-(1/T))
        e_si=e_si0*np.exp(inner)
        return e_si
    ei_s = IceSaturationVaporPressure(T)
    H_ice = e / ei_s

    InputDictionary = {'P': P, 'p0': p0, 'rv': rv, 'rt': rt,
                       'Rd': Rd, 'Rv': Rv, 'Cpd': Cpd, 'Cpv': Cpv, 'Cpl': Cpl, 
                       'Lv': Lv, 'T': T, 'e': e, 'Pd': Pd,
                       'e_s': e_s, 'H_vapor': H_vapor, 'H_ice': H_ice
                       }
    return InputDictionary

def VariableCalculation(inputDictionary):
    [P,p0,rv,rt] = (inputDictionary[k] for k in ["P","p0","rv","rt"])
    [Rd,Rv,Cpd,Cpv,Cpl] = (inputDictionary[k] for k in ["Rd","Rv","Cpd","Cpv","Cpl"])
    [Lv,T,e,Pd] = (inputDictionary[k] for k in ["Lv","T","e","Pd"])
    [e_s,H_vapor,H_ice] = (inputDictionary[k] for k in ["e_s","H_vapor","H_ice"])

    ################################# EQUIVALENT POTENTIAL TEMPERATURE
    divisor=(Cpd+Cpl*rt)
    theta_e=(T*(p0/Pd)**(Rd/divisor))*(H_vapor**(-rv*Rv/divisor))*np.exp(Lv(T)*rv/(divisor*T))

    outputDictionary={'theta_e': theta_e,
                      'RH_vapor': H_vapor,
                      'RH_ice': H_ice}
    return outputDictionary

In [10]:
####################################
#RUNNING

In [11]:
#CALCULATING AND APPENDING TO DATA EACH TIMESTEP
for t in num_elements:
    if np.mod(t,1)==0: print(f'Current time {t}')

    #getting timestring for loading input data
    timeString = ModelData.timeStrings[t]

    #loading input variables
    inputDictionary = GetInputVariables(DataManager.inputDataDirectory, timeString)

    #calculating variables
    outputDictionary = VariableCalculation(inputDictionary)
    
    #outputting
    DataManager.SaveOutputTimestep(DataManager.outputDataDirectory, timeString, outputDictionary)

Current time 0
Saved timestep to output file: /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Variable_Calculation/CalculateMoreVariables/1km_5min_34nz/Equivalent_Potential_Temperature/Equivalent_Potential_Temperature_1km_5min_34nz_0-00-00.h5
Current time 1
Saved timestep to output file: /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Variable_Calculation/CalculateMoreVariables/1km_5min_34nz/Equivalent_Potential_Temperature/Equivalent_Potential_Temperature_1km_5min_34nz_0-05-00.h5
Current time 2
Saved timestep to output file: /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Variable_Calculation/CalculateMoreVariables/1km_5min_34nz/Equivalent_Potential_Temperature/Equivalent_Potential_Temperature_1km_5min_34nz_0-10-00.h5
Current time 3
Saved timestep to output file: /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Variable_Calculation/CalculateMor

In [ ]:
######################################################

In [ ]:
# #READING BACK IN
# def GetVariableData(t, dataName="Virtual_Potential_Temperature"):
#     res = ModelData.res
#     t_res = ModelData.t_res
#     Nz_str = ModelData.Nz_str
#     inputDirectory = os.path.join(DataManager.outputDirectory, f"{res}_{t_res}_{Nz_str}nz", dataName)
#     timeString = ModelData.timeStrings[t]

#     FileName = os.path.join(inputDirectory, f"{dataName}_{res}_{t_res}_{Nz_str}nz_{timeString}.h5")

#     dataDictionary = {}
#     with h5py.File(FileName, 'r') as f:
#         print("Keys in file:", list(f.keys()))
#         for key in f.keys():
#             dataDictionary[key] = f[key][:]
#             print(f"{key}: shape = {dataDictionary[key].shape}, dtype = {dataDictionary[key].dtype}")
#     return dataDictionary

# dataDictionary = GetVariableData(t=0)